# Step3-2nd preprocessing

## 1. Impot necessary modules & start a spark session

In [ ]:
# Import necessary modules
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import matplotlib.pyplot as plt
import os

In [ ]:
# Create a Spark session
spark = (
    SparkSession.builder.appName('ADS_project_1.py')
    .config('spark.sql.repl.eagerEval.enabled', True)
    .config('spark.sql.parquet.cacheMetadata', 'true')
    .config('spark.sql.session.timeZone', 'Etc/UTC')
    .config('spark.driver.memory', '16g')
    .config('spark.executer.memory', '16g')
    .getOrCreate()
)

## 2. Data import & overview

Import raw `TLC_data` from directory `data/raw/TLC_data/`

In [ ]:
TLC_data_path = '../data/raw/TLC_data/TLC_data.parquet/'
TLC_data = spark.read.parquet(TLC_data_path)

Show #rows, #cols & overview of `TLC_data`

In [ ]:
num_rows_after_1st_preprocessing = TLC_data.count()
num_cols_after_1st_preprocessing = len(TLC_data.columns)

print('number of rows: ', num_rows_after_1st_preprocessing)
print('number of cols: ', num_cols_after_1st_preprocessing)
TLC_data.limit(5)

## 3. Remove features with invalid values

Descriptive Statistics of each feature

In [ ]:
# Getting the summary statistics of the entire dataset
TLC_data.describe()

We notice there exists invalid values in the features '#passenger', 'trip_distance' and 'extra_fee', then remove them

In [ ]:
TLC_data = TLC_data.where(F.col('#passenger') > 0) \
                   .where(F.col('trip_distance') > 0) \
                   .where(F.col('extra_fee') > 0)

We notice that all the values in the feature 'airport_fee' are `null`, then remove the feature

In [ ]:
TLC_data = TLC_data.drop('airport_fee')

We notice there exists a lot of `null` in the feature 'congestion_fee', then change them to `0.0`

In [ ]:
TLC_data = TLC_data.fillna({"congestion_fee": 0.0})

Show data shape

In [ ]:
print('number of rows: ', TLC_data.count())
print('number of cols: ', len(TLC_data.columns))
TLC_data.limit(5)

## 4. Feature engineering

### 4.1 Create new features

Create new features for further analysis: 'date', 'trip_duration', 'average_speed', 'if_weekend', 'if_peak_hour', 'if_overnight', 'if_airport'

In [ ]:
TLC_data = (
    TLC_data
        # Create 'date' from 'pickup_time' by extracting month and day
        .withColumn('date', F.date_format('pickup_time', 'MM-dd'))

        # Create 'trip_duration' from 'dropoff_time' & 'pickup_time', in unit (s)
        .withColumn('trip_duration', F.unix_timestamp('dropoff_time') - F.unix_timestamp('pickup_time'))

        # Create 'average_speed' from 'trip_distance' & 'trip_duration', in unit (miles/h)
        .withColumn('average_speed', F.col('trip_distance') / (F.col('trip_duration') / 3600))
        .withColumn('average_speed', F.col('average_speed').cast('float'))
        .withColumn('average_speed', F.round(F.col('average_speed'), 2))

        # Create 'if_weekend' from 'pickup_time', values are 0 & 1
        .withColumn('if_weekend', F.dayofweek('pickup_time').isin([1, 7]).cast(IntegerType()))

        # Create 'if_morning_peak' from 'pickup_time', values are 0 & 1
        .withColumn('if_morning_peak', (F.hour('pickup_time').between(7, 10)).cast(IntegerType()))

        # Create 'if_evening_peak' from 'pickup_time', values are 0 & 1
        .withColumn('if_evening_peak', (F.hour('pickup_time').between(16, 19)).cast(IntegerType()))

        # Create 'if_peak_hour' from 'if_morning_peak' & 'if_evening_peak', values are 0 & 1
        .withColumn('if_peak_hour', F.expr("if_morning_peak = 1 OR if_evening_peak = 1").cast(IntegerType()))

        # Create 'if_overnight' from 'pickup_time', values are 0 & 1
        .withColumn('if_overnight', (F.hour('pickup_time').isin([23, 0, 1, 2, 3, 4, 5, 6]).cast(IntegerType())))

        # Create 'if_airport' from 'up_location_id' & 'off_location_id', values are 0 & 1
        .withColumn(
            'if_airport',
            ((F.col('up_location_id')).isin([1, 132, 138]) | (F.col('off_location_id')).isin([1, 132, 138]))
            .cast(IntegerType())
        )
)

TLC_data.limit(5)

Show data shape

In [ ]:
print('number of rows: ', TLC_data.count())
print('number of cols: ', len(TLC_data.columns))

### 4.2 Remove invalid values in the new features

Descriptive Statistics of 2 new features, which may exists invalid values

In [ ]:
TLC_data.describe('trip_duration', 'average_speed')

We notice there exists invalid values in the features 'trip_duration' & 'average_speed', then remove them

In [ ]:
TLC_data = TLC_data.where(F.col('trip_duration') > 0) \
                   .where(F.col('average_speed') > 0)

### 4.3 Changes for readability

Delete features that have already been used to extract information and no longer needed

In [ ]:
used_feature_list = ['pickup_time', 'dropoff_time', 'extra_fee', 'if_morning_peak', 'if_evening_peak']

for used_feature in used_feature_list:
    TLC_data = TLC_data.drop(used_feature)

Reorder the features for readability

In [ ]:
TLC_data = TLC_data.select(
    'date', 'up_location_id', 'off_location_id', '#passenger', 'trip_distance', 'trip_duration', 'average_speed', 
    'congestion_fee', 'toll_fee', 'if_weekend', 'if_peak_hour', 'if_overnight', 'if_airport'
)

Show data shape after creating new features and removing some invalid values

In [ ]:
print('number of rows: ', TLC_data.count())
print('number of cols: ', len(TLC_data.columns))
TLC_data.limit(5)

## 5. Remove outliers

### 5.1 sample `TLC_data` for box plot

In [ ]:
# Define the sample size and sample from TLC_data
sample_size = 0.01
sample_TLC_data = TLC_data.sample(sample_size, seed=1).toPandas()
print('#rows_of_sample_data: ', len(sample_TLC_data))

### 5.2 Plot a box plot to detect outliers

Define a list of features, which may exists outliers

In [ ]:
figure_feature_list = ['trip_distance', 'trip_duration', 'average_speed', 'congestion_fee', 'toll_fee']

Plot the box plot for detecting outlier

In [ ]:
# Plot boxplots for the features in the list
fig, axs = plt.subplots(1, 5, figsize=(15,5))
for i in range(0, 5):
    axs[i].boxplot(sample_TLC_data[figure_feature_list[i]])
    axs[i].set_xlabel(figure_feature_list[i], size = 16)
fig.suptitle(f"Boxplot", fontsize=18)

plt.savefig('../plots/boxplot_for_detecting_outlier_1')
plt.show()

congestion_fee并没有outlier，所以不进行移除outlier

We find that there's no oulier in the feature 'congestion_fee', so we decide not to remove outliers for this feature

At first we did remove outliers for 'toll_fee' as well, but after replotting the box plot we find that all values except 0 are removed. We thought the definition of outliers is too strict for this feature, so we decided not to remove outliers for 'toll_fee'.

### 5.3 Remove outliers using IQR method

Redefine the features which outliers should be removed from

In [ ]:
figure_feature_list = ['trip_distance', 'trip_duration', 'average_speed']

Remove outliers using IQR method

In [ ]:
# Loop through each feature in figure_feature_list and remove outliers
for feature in figure_feature_list:
    quantiles = TLC_data.approxQuantile(feature, [0.25, 0.75], 0.05)
    Q1 = quantiles[0]
    Q3 = quantiles[1]
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    TLC_data = TLC_data.filter((F.col(feature) >= lower_bound) & (F.col(feature) <= upper_bound))

### 5.4 Replot the box plot after removing outliers

重新取样并绘制移除完outlier之后的箱形图

Resample from `TLC_data` and replot

In [ ]:
# Resample with sample_size = 0.01
sample_size = 0.01
sample_TLC_data = TLC_data.sample(sample_size, seed=1).toPandas()

# Replot
fig, axs = plt.subplots(1, 3, figsize=(15,5))
for i in range(0, 3):
    axs[i].boxplot(sample_TLC_data[figure_feature_list[i]])
    axs[i].set_xlabel(figure_feature_list[i], size = 16)
fig.suptitle(f"Boxplot", fontsize=18)

plt.savefig('../plots/boxplot_for_detecting_outlier_2')
plt.show()

The box plot looks much better

## 6. Data overview after 2nd preprocessing & saving

`TLC_data` overview after 2nd preprocessing

In [ ]:
num_rows_after_2nd_preprocessing = TLC_data.count()
num_cols_after_2nd_preprocessing = len(TLC_data.columns)

num_removed_rows = num_rows_after_1st_preprocessing - num_rows_after_2nd_preprocessing

print('number of rows: ', num_rows_after_2nd_preprocessing)
print('number of cols: ', num_cols_after_2nd_preprocessing)
print('\n')
print('number of removed rows: ', num_removed_rows)

Save `TLC_data` to directory `data/curated/TLC_data/`

In [ ]:
# Define the directory for saving 2nd preprocessed data
directory = ('../data/curated/TLC_data')
# Check if the directory exists; if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)

# Save TLC_data
TLC_data.write.mode('overwrite').parquet('../data/curated/TLC_data/TLC_data.parquet')

## 7. Stop spark session

In [ ]:
spark.stop()